# 1. Import and Install Dependencies

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time , random
import mediapipe as mp
import copy
import glob
from sklearn.model_selection import train_test_split
import keras 
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense,Dropout,BatchNormalization,Input,Conv1D,MaxPooling1D,\
                                    TimeDistributed,Activation,Lambda,ReLU,Conv1D,ConvLSTM1D,Flatten
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint,EarlyStopping
from tensorflow.keras import layers
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score,confusion_matrix
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.python.client import device_lib 
from tensorflow.keras.utils import plot_model
import seaborn as sns

In [3]:
# Check GPU Existing
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18392351820812195056
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4922343424
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8180013878408938606
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


# 2. Keypoints using MP Holistic

In [4]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [5]:
def mediapipe_detection(image, model):
    """
    inputs: CV2 Image
    output: Image, detected Landmarks
    """
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable to Improve Perf.
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [6]:
# Draw Landmarks on Image
def draw_landmarks(image, results):
    """
    inputs: CV2 Image , Landmarks[Model Results]
    """
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [7]:
# Draw Styled Landmarks on Image
def draw_styled_landmarks(image, results):
    """
    inputs: CV2 Image , Landmarks[Model Results]
    """
    # Draw face connections

    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [7]:
# Extract Keypoints from Landmarks And Concatenate it in One Array
def extract_keypoints(results):
    """
    inputs: Resutls from MediaPipe Model
    output: Concatenated Landmarks
    """
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh]) 

# 4. Setup Folders for Collection

In [8]:
# Define our Actions
fifty_actions = np.array(['All',
'Boy',
'Girl',
'Book',
'Mobile',
'Car',
'Baby',
'Door',
'End',
'Eat',
'Drink',
'Father',
'Mother',
'Go',
'Good',
'Bad',
'Food',
'House',
'In',
'Out',
'Sad',
'Happy',
'Man',
'Woman',
'Stop',
'School',
'New',
'Old',
'Play',
'Room',
'See',
'Sit',
'Sister',
'Brother',
'Think',
'Work',
'name',
'Yes',
'No',
'Walk',
'Love',
'Need',
'Respect',
'Money',
'I',
'You',
'Day',
'Ambulance',
'Buy',
'Bread',])
sorted_actions=sorted(fifty_actions)
ten_actions=np.array(['Behind','Brother','Egypt','Father','Front of','Hello','How are you','I','Mother'
,'Sister'])



# 5. Preprocess Data and Create Labels and Features

In [9]:
# Labeling Actions to Numbers
label_map = {label:num for num, label in enumerate(sorted_actions)}

In [10]:
label_map

{'All': 0,
 'Ambulance': 1,
 'Baby': 2,
 'Bad': 3,
 'Book': 4,
 'Boy': 5,
 'Bread': 6,
 'Brother': 7,
 'Buy': 8,
 'Car': 9,
 'Day': 10,
 'Door': 11,
 'Drink': 12,
 'Eat': 13,
 'End': 14,
 'Father': 15,
 'Food': 16,
 'Girl': 17,
 'Go': 18,
 'Good': 19,
 'Happy': 20,
 'House': 21,
 'I': 22,
 'In': 23,
 'Love': 24,
 'Man': 25,
 'Mobile': 26,
 'Money': 27,
 'Mother': 28,
 'Need': 29,
 'New': 30,
 'No': 31,
 'Old': 32,
 'Out': 33,
 'Play': 34,
 'Respect': 35,
 'Room': 36,
 'Sad': 37,
 'School': 38,
 'See': 39,
 'Sister': 40,
 'Sit': 41,
 'Stop': 42,
 'Think': 43,
 'Walk': 44,
 'Woman': 45,
 'Work': 46,
 'Yes': 47,
 'You': 48,
 'name': 49}

## Load All Data 

In [15]:
# Load Data 60 fps then Transform it to 30fps
sequences1, labels1 = [], []
DATA_PATH_KEYPOINTS = os.path.join(r'D:\signara\Signara-main\Sign To Text\utils\MP_Enhanced_Data_KEYPOINTS')
for action in fifty_actions:
    list_seq = glob.glob('/'.join([DATA_PATH_KEYPOINTS,action,'*']))
    for sequence in list_seq:       
        window = []
        last_frame=[]
        # Take two Steps to Convert it To 30FPS
        for frame_num in range(0,60,2):
            #Load Frame Keypoints
            if os.path.exists(os.path.join(sequence, "{}.npy".format(frame_num))):
                
                res1 = np.load(os.path.join(sequence, "{}.npy".format(frame_num)))
                #Take Hands Landmarks
                lh_rh = res1[1536:]
                #Remove z Axis From Landmarks
                for z in range(2,lh_rh.shape[0],3):
                        lh_rh[z] = None
                #Remove NaN Data
                lh_rh = lh_rh[np.logical_not(np.isnan(lh_rh))]
                is_all_zero = np.all((lh_rh == 0))
                if is_all_zero:
                    if len(last_frame)==0:
                        window.append([0 for i in range(84)])
                        continue
                    window.append(last_frame)
                    continue
                last_frame=lh_rh
                window.append(lh_rh)
            else:
                if len(last_frame)==0:
                    window.append([0 for i in range(84)])
                    continue
                window.append(last_frame)
            
            
        sequences1.append(window)
        labels1.append(label_map[action])

In [24]:
#Convert Lists To Array
X2 = np.array(sequences1)
#Convert Labels to OHE
y2 = to_categorical(labels1).astype(int)
X2.shape, y2.shape


((2250, 30, 84), (2250, 50))

In [ ]:
# Load 30 Fps Data Folder 
DATA_PATH_KEYPOINTS = os.path.join(r'D:\signara\Signara-main\Sign To Text\utils\AllData')
sequences, labels = [], []
for action in ten_actions:
    list_seq = glob.glob('/'.join([DATA_PATH_KEYPOINTS,'30FPS',action,'*']))
    for sequence in list_seq:       
        window = []
       
        for frame_num in range(0,30):
            if os.path.exists(os.path.join(sequence, "{}.npy".format(frame_num))):
                #Load Frame Keypoints
                res1 = np.load(os.path.join(str(sequence), "{}.npy".format(frame_num)))
                #Take Hands Landmarks
                lh_rh = res1[1536:]
                #Remove z Axis From Landmarks
                for z in range(2,lh_rh.shape[0],3):
                        lh_rh[z] = None
                #Remove NaN Data
                lh_rh = lh_rh[np.logical_not(np.isnan(lh_rh))]
                is_all_zero = np.all((lh_rh == 0))

                window.append(lh_rh)
            else:
                window.append([0 for i in range(84)])
            
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
#Convert Lists To Array
X1 = np.array(sequences)
#Convert Labels to OHE
y1 = to_categorical(labels).astype(int)
X1.shape, y1.shape

((570, 30, 84), (570, 10))

In [15]:
# concatenate all data
X = np.concatenate([X1,X2])
y = np.concatenate([y1,y2])
X.shape,y.shape

((1110, 30, 84), (1110, 10))

In [16]:
#save data for ease of use
np.save(r'D:\signara\Signara-main\Sign To Text\utils\X_data',X)
np.save(r'D:\signara\Signara-main\Sign To Text\utils\y_data',y)


In [16]:
# load data from saved file 
X=np.load(r'D:\signara\Signara-main\Sign To Text\utils\X_data.npy')
y=np.load(r'D:\signara\Signara-main\Sign To Text\utils\y_data.npy')
X.shape,y.shape

((1110, 30, 84), (1110, 10))

## Data Augmentation

In [27]:
# Rotation Augmentation
def augment_data_rotataion(X,y):
    '''
    input: X,y  as numpy array Shape: [Samples,Timesteps,Features]
    output: Augmented X,y as numpy array Shape:[Samples,Timesteps,Features]
    '''
    # Make an Array with Shape Like Original One
    augmented_X = np.zeros_like(X)
    augmented_y = np.zeros_like(y)
    
    #Looping in all Examples
    for ex in range(X.shape[0]):
        # Get Random Angle Betwwen -5,5
        rotation_angle = random.randint(-5,5)
        # Convert it to Radians
        theta = np.radians(rotation_angle)
        c, s = np.cos(theta), np.sin(theta)
        # Build a Rotation Matrix
        rotation_matrix = np.array(((c, -s), (s, c)))
        # Looping Each Frame
        for frame in range(X.shape[1]):
            window = []
            # looping each Point within Frame
            for i in range(0,X.shape[2]-1,2):
                # Get Keypoint
                keypoint = np.array([X[ex][frame][i],X[ex][frame][i+1]])
                # Calculate Rotated Keypoint
                rotated_keypoint = np.dot(rotation_matrix, keypoint)
                keypoint_x = rotated_keypoint[0]
                keypoint_y = rotated_keypoint[1]
                # Append New Keypoint To our Data
                window.extend([keypoint_x,keypoint_y])
            augmented_X[ex][frame] = np.array(window)
        augmented_y[ex] = y[ex]
    return augmented_X,augmented_y

In [28]:
# Scale Augmentation
def augment_data_scale(X,y):
    '''
    input: X,y  as numpy array Shape: [Samples,Timesteps,Features]
    output: Augmented X,y as numpy array Shape: [Samples,Timesteps,Features]
    '''
    # Make an Array with Shape Like Original One
    augmented_X = np.zeros_like(X)
    augmented_y = np.zeros_like(y)
    # Looping in Each Sample
    for ex in range(X.shape[0]):
        # Get Random Scale Factor
        SCALE = round(random.random(),2)
        for frame in range(X.shape[1]):
            # Calculate New Point
            augmented_X[ex][frame] = X[ex][frame]*SCALE
        augmented_y[ex] = y[ex]
    return augmented_X,augmented_y

In [29]:
# Augmented Rotated Data
rot_x,rot_y = augment_data_rotataion(X2,y2) 
# Augmented Scaled Data
scaled_x,scaled_y = augment_data_scale(X2,y2) 

In [31]:
# concatenate all data [Original and Augmented]
X_ = np.concatenate([X2,rot_x,scaled_x])
y_ = np.concatenate([y2,rot_y,scaled_y])
X_.shape,y_.shape

((6750, 30, 84), (6750, 50))

In [32]:
# save data in .npy files for ease of usage
np.save(r'D:\signara\Signara-main\Sign To Text\utils\X50_aug_data',X_)
np.save(r'D:\signara\Signara-main\Sign To Text\utils\y50_aug_data',y_)


In [11]:
X_=np.load(r'D:\signara\Signara-main\Sign To Text\utils\X50_aug_data.npy')
y_=np.load(r'D:\signara\Signara-main\Sign To Text\utils\y50_aug_data.npy')
X_.shape,y_.shape

((6750, 30, 84), (6750, 50))

In [12]:
# Split Data
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.20,shuffle=True,stratify=y_,random_state=42)

In [13]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5400, 30, 84), (1350, 30, 84), (5400, 50), (1350, 50))

# 6. Build and Train Neural Network Models

In [14]:
# Define The Time Steps
timesteps = X_.shape[1]
# Define The Features No.
features = X_.shape[2]

## 6.1 LSTM

In [ ]:
# Build a LSTM Model Arch
model_lstm = Sequential()
model_lstm.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(timesteps,features))) # frames * Features
model_lstm.add(LSTM(128, return_sequences=True, activation='relu'))
model_lstm.add(LSTM(64, return_sequences=False, activation='relu'))
model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dense(32, activation='relu'))
model_lstm.add(Dense(actions.shape[0], activation='softmax'))
model_lstm.summary()

In [ ]:
# Compilation Configuration
model_lstm.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
# Define Callbacks
log_dir = os.path.join('Logs/LSTM3')
tb_callback = TensorBoard(log_dir=log_dir,histogram_freq=1,
                          update_freq='epoch',
                          profile_batch=0) ## !tensorboard --logdir=.
mc = ModelCheckpoint('Models/LSTM3.h5', monitor='val_categorical_accuracy', mode='max', verbose=1,save_best_only=True)
es = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1,patience=100)
callbacks = [tb_callback,mc,es]

In [ ]:
model_lstm.fit(X_train, y_train, epochs=2000, callbacks=[callbacks],batch_size=32,validation_data=(X_test,y_test),initial_epoch = 13)

## 6.2 Conv1d + LSTM

In [16]:
model_convlstm = Sequential()

model_convlstm.add(Input(shape=(timesteps,features))) 
model_convlstm.add(Conv1D(128, # Filters
                 5, # Kernel Size
                 padding='same',
                 activation='relu',
                 strides=1))
model_convlstm.add(MaxPooling1D(pool_size=4))
model_convlstm.add(LSTM(64, return_sequences=False))
model_convlstm.add(Dense(32, activation='relu'))
model_convlstm.add(Dense(len(sorted_actions)))



model_convlstm.summary()

: 

: 

In [ ]:
log_dir = os.path.join('Logs/Conv1d_LSTM_custom')
tb_callback = TensorBoard(log_dir=log_dir,histogram_freq=1,
                          update_freq='epoch',
                          profile_batch=0) ## !tensorboard --logdir=.
mc = ModelCheckpoint('Models/Conv1d_LSTM_custom.h5', monitor='val_categorical_accuracy', mode='max', verbose=1,save_best_only=True)
es = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1,patience=50)
callbacks = [tb_callback,mc,es]

In [ ]:
model_convlstm.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model_convlstm.fit(X_train, y_train, epochs=2000, callbacks=[callbacks],batch_size=4,validation_data=(X_test,y_test))

## 6.3 CONVLSTM1D TensorFlow

In [ ]:
# (samples, time, rows, channels)
X_train_reshaped = X_train.reshape([X_train.shape[0],X_train.shape[1],X_train.shape[2],1])
X_test_reshaped = X_test.reshape([X_test.shape[0],X_train.shape[1],X_train.shape[2],1])
X_train_reshaped.shape,X_test_reshaped.shape

In [ ]:
model_conv1d_lstm_tf = Sequential()

model_conv1d_lstm_tf.add(Input(shape=(X_train.shape[1],X_train.shape[2],1))) # time steps(frames) * features
model_conv1d_lstm_tf.add(ConvLSTM1D(filters=64,kernel_size=(5), data_format='channels_last', padding = 'same'
                      ,return_sequences=True))
model_conv1d_lstm_tf.add(ConvLSTM1D(filters=32,kernel_size=(5), data_format='channels_last', padding = 'same'
                      ,return_sequences=False))
model_conv1d_lstm_tf.add(MaxPooling1D((32)))
#model2.add(Dense(32, activation='relu'))
model_conv1d_lstm_tf.add(TimeDistributed(Dense(actions.shape[0], activation='relu')))
model_conv1d_lstm_tf.add(Flatten())
model_conv1d_lstm_tf.add(Dense(actions.shape[0]))



model_conv1d_lstm_tf.summary()

In [ ]:
log_dir = os.path.join('Logs/Conv1d_LSTM_tf')
tb_callback = TensorBoard(log_dir=log_dir,histogram_freq=1,
                          update_freq='epoch',
                          profile_batch=0) ## !tensorboard --logdir=.
mc = ModelCheckpoint('Models/model_conv1d_lstm_tf.h5', monitor='val_categorical_accuracy', mode='max', verbose=1,save_best_only=True)
es = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1,patience=50)
callbacks = [tb_callback,mc,es]

In [ ]:
model_conv1d_lstm_tf.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model_conv1d_lstm_tf.fit(X_train_reshaped, y_train,callbacks=[callbacks], epochs=2000, batch_size=1,validation_data=(X_test_reshaped,y_test))

## 6.4 Conv1d

In [ ]:
model_conv1d = Sequential()
model_conv1d.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1],X_train.shape[2])))
model_conv1d.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model_conv1d.add(Dropout(0.5))
model_conv1d.add(MaxPooling1D(pool_size=2))
model_conv1d.add(Flatten())
model_conv1d.add(Dense(100, activation='relu'))
model_conv1d.add(Dense(actions.shape[0], activation='softmax'))
model_conv1d.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_conv1d.summary()

In [ ]:
log_dir = os.path.join('Logs/Conv1d')
tb_callback = TensorBoard(log_dir=log_dir,histogram_freq=1,
                          update_freq='epoch',
                          profile_batch=0) ## !tensorboard --logdir=.
mc = ModelCheckpoint('Models/model_conv1d.h5', monitor='val_categorical_accuracy', mode='max', verbose=1,save_best_only=True)
es = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1,patience=50)
callbacks = [tb_callback,mc,es]

In [ ]:
model_conv1d.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
model_conv1d.fit(X_train, y_train, epochs=2000, callbacks=[callbacks],batch_size=1,validation_data=(X_test,y_test))

## 6.5 With CTC Loss & Decoder

In [ ]:
class CTCLayer(layers.Layer):
    def __init__(self, name=None):
        super().__init__(name=name)
        self.loss_fn = keras.backend.ctc_batch_cost

    def call(self, y_true, y_pred):
        # Compute the training-time loss value and add it
        # to the layer using `self.add_loss()`.
        batch_len = tf.cast(tf.shape(y_true)[0], dtype="int64")
        input_length = tf.cast(tf.shape(y_pred)[1], dtype="int64")
        label_length = tf.cast(tf.shape(y_true)[1], dtype="int64")

        input_length = input_length * tf.ones(shape=(batch_len, 1), dtype="int64")
        label_length = label_length * tf.ones(shape=(batch_len, 1), dtype="int64")

        loss = self.loss_fn(y_true, y_pred, input_length, label_length)
        self.add_loss(loss)

        # At test time, just return the computed predictions
        return y_pred

In [ ]:
def build_model():

    input_seq = layers.Input(
            shape=(timesteps, features), name="seq", dtype="float32"
        )
    labels = layers.Input(name="label", shape=(None,), dtype="int32")

    x = layers.Bidirectional(layers.LSTM(128, return_sequences=True, dropout=0.25))(input_seq)
    #x = layers.BatchNormalization(name="lstm_1_bn")(x)
    #x = layers.ReLU(name="lstm_1_relu")(x)
    x = layers.Bidirectional(layers.LSTM(64, return_sequences=True, dropout=0.25))(x)
    #x = layers.BatchNormalization(name="lstm_2_bn")(x)
    #x = layers.ReLU(name="lstm_2_relu")(x)
    #x = Dense(64, activation='relu')(x)

    # Output layer
    x = layers.Dense(actions.shape[0]+1, activation="softmax", name="dense2")(x)

    # Add CTC layer for calculating CTC loss at each step
    output = CTCLayer(name="ctc_loss")(labels, x)

    # Define the model
    model = keras.models.Model(
        inputs=[input_seq, labels], outputs=output, name="ctc_model_v1"
    )
    # Optimizer
    # Compile the model and return
    model.compile(optimizer='adam')
    return model

In [ ]:
ctc_model = build_model()
ctc_model.summary()

In [ ]:
def split_data(images, labels, train_size=0.9, shuffle=True):
    # 1. Get the total size of the dataset
    size = len(images)
    # 2. Make an indices array and shuffle it, if required
    indices = np.arange(size)
    if shuffle:
        np.random.shuffle(indices)
    # 3. Get the size of training samples
    train_samples = int(size * train_size)
    # 4. Split data into training and validation sets
    x_train, y_train = images[indices[:train_samples]], labels[indices[:train_samples]]
    x_valid, y_valid = images[indices[train_samples:]], labels[indices[train_samples:]]
    return x_train, x_valid, y_train, y_valid


# Splitting data into training and validation sets
x_train, x_valid, y_train, y_valid = split_data(np.array(X2), np.array(labels1))

In [ ]:
def encode_single_sample(seq, label):
    return {"seq": seq, "label": label}

In [ ]:
batch_size = 1
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = (
    train_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE
    )
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

validation_dataset = tf.data.Dataset.from_tensor_slices((x_valid, y_valid))
validation_dataset = (
    validation_dataset.map(
        encode_single_sample, num_parallel_calls=tf.data.AUTOTUNE
    )
    .batch(batch_size)
    .prefetch(buffer_size=tf.data.AUTOTUNE)
)

In [ ]:
epochs = 500
early_stopping_patience = 300
# Add early stopping
early_stopping = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=early_stopping_patience, restore_best_weights=True
)
log_dir = os.path.join('Logs/CTC')
tb_callback = TensorBoard(log_dir=log_dir) ## !tensorboard --logdir=.
# Train the model
history = ctc_model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    callbacks=[early_stopping,tb_callback],
)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

### Inference

In [ ]:
prediction_model = keras.models.Model(
    ctc_model.get_layer(name="seq").input, ctc_model.get_layer(name="dense2").output
)
prediction_model.summary()

In [ ]:
max_length = 1

In [ ]:
# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    input_len = np.ones(pred.shape[0]) * pred.shape[1]
    # Use greedy search. For complex tasks, you can use beam search
    results = keras.backend.ctc_decode(pred, input_length=input_len, greedy=True)[0][0][
        :, :max_length
    ]
    prob = np.array(keras.backend.ctc_decode(pred, input_length=input_len, greedy=True,)[1][0][0]) #[0])#[:, :19])
    # Iterate over the results and get back the text
    output_text = []
    for res in results:
        #res = tf.strings.reduce_join(res).numpy().decode("utf-8")
        output_text.append(res)
    return output_text , prob


In [ ]:
#  Let's check results on some validation samples
predss=[]
for batch in validation_dataset.take(10000):
    batch_seq = batch["seq"]
    batch_labels = batch["label"]
    
    preds = prediction_model.predict(batch_seq)
    pred_texts = decode_batch_predictions(preds)[0]
    predss.append((pred_texts == batch_labels)[0][0].numpy())

In [ ]:
sum(predss)/len(predss)

## 6.6 Transformer

In [16]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [17]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(actions.shape[0], activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [18]:
input_shape = X_train.shape[1:]

In [19]:
model_transformer = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)
model_transformer.summary()

: 

: 

In [ ]:
model_transformer.compile(
    loss="categorical_crossentropy",
    optimizer=Adam(learning_rate=1e-5),
    metrics=["categorical_accuracy"],
)

In [ ]:
log_dir = os.path.join('Logs/transformer_encoder')
tb_callback = TensorBoard(log_dir=log_dir,histogram_freq=1,
                          update_freq='epoch',
                          profile_batch=0) ## !tensorboard --logdir=.
mc = ModelCheckpoint('Models/model_transformer_encoder.h5', monitor='val_categorical_accuracy', mode='max', verbose=1,save_best_only=True)
es = EarlyStopping(monitor='val_categorical_accuracy', mode='max', verbose=1,patience=100)
callbacks = [tb_callback,mc,es]

In [1]:
model_transformer=tf.keras.models.load_model(r'D:\signara\Signara-main\Sign To Text\utils\model')

NameError: name 'tf' is not defined

In [ ]:
model_transformer.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=2000,
    batch_size=8,
    callbacks=callbacks,
)

In [ ]:
model_transformer.evaluate(X_test, y_test, verbose=1)


In [ ]:
np.argmax(model_transformer.predict(np.expand_dims(X_test[87],axis=0)))

In [ ]:
np.argmax(y_test[87])

# 7. Make Predictions

In [ ]:
res = model.predict(X_test)
np.argmax(res)

In [ ]:
actions[np.argmax(res[4])]

In [ ]:
actions[np.argmax(y_test[4])]

# 8. Save and Load Weights

In [ ]:
model.save('action.h5')

In [ ]:
del model

In [ ]:
model.load_weights('model_transformer_encoder.h5')

# 9. Evaluation using Confusion Matrix and Accuracy

In [ ]:
yhat = model_lstm.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
actions

In [ ]:
sns.heatmap(confusion_matrix(ytrue, yhat),annot =True,yticklabels=actions,xticklabels=actions)

In [ ]:
accuracy_score(ytrue, yhat)